In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import re
import pickle as pk
import numpy as np
import keras as kr
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional

In [29]:
pd.set_option('display.max_colwidth',-1)
df= pd.read_excel("text")
df= df.loc[:,"desc1":"Assignment group"]

## Cleaning the text using Regular Expressions

In [30]:
df["desc1"] = df["desc1"].fillna('').apply(str)
df["desc1"] = df["desc1"].str.lower()
df["desc1"] = df["desc1"].apply(lambda x: re.sub('http\S+:\/\/.*','',x))
df["desc1"] = df["desc1"].apply(lambda x: re.sub('\w*\d\w*"','',x))
df["desc1"] = df["desc1"].str.replace('[^\w\s]','')
df["desc1"] = df["desc1"].str.replace('\d+', '')

## Removing stop words

In [ ]:
list2= ['words which you want to remove']
list1= list(stopwords.words('english'))
list3= list1+list2
df["description"]=df["description"].apply(lambda x: ' '.join([word for word in x.split() if word not in (list3)]))

## Removing un common words

In [31]:
freq=pd.Series(' '.join(df["desc1"]).split()).value_counts()[-3000:]

In [33]:
df["desc1"] = df["desc1"].apply(lambda x: ' '.join(x for x in x.split() if x not in freq))
df["desc1"] = df["desc1"].apply(lambda x:" ".join(x for x in x.split() if len(x) < 12))
df["desc1"] = df["desc1"].apply(lambda x:" ".join(x for x in x.split() if len(x) > 3))

## Tokenization

In [34]:
tokenizer = Tokenizer(num_words =8000,oov_token='<OOV>')
tokenizer.fit_on_texts(df["desc1"])
word_index = tokenizer.word_index
dict1=dict(list(word_index.items()))

### Saving the dictinary of tokens in JSON and EXCEL file

In [29]:
#import json

#with open('dict1.json', 'w') as fp:
#    json.dump(dict1, fp)

In [30]:
#list1 = [(k, v) for k, v in dict1.items()] 
#ne=pd.DataFrame(list1)
#ne.to_excel("C:/Users/PC-ASUS/Desktop/ITIL final files/dictionary.xlsx")

### Loading the dictionary

In [31]:
#import json
#with open('dict1.json', 'r') as fp:
#    dict1 = json.load(fp)

In [32]:
#ne= pd.read_excel("C:/Users/PC-ASUS/Desktop/ITIL final files/dictionary.xlsx")

### Saving and loading the Tokenizer object

In [6]:
with open('tokenizer.pickle', 'wb') as handle:
    pk.dump(tokenizer, handle, protocol=pk.HIGHEST_PROTOCOL)

In [35]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pk.load(handle)

In [36]:
word_index = tokenizer.word_index
dict1=dict(list(word_index.items()))

### Text to Sequence

In [37]:
tokenizer.fit_on_texts(df["desc1"])
X = tokenizer.texts_to_sequences(df["desc1"])
X= pad_sequences(X, maxlen=1000, padding="post", truncating="post")

## Encoding Function

In [5]:
def encoding(text):
    tokens=kr.preprocessing.text.text_to_word_sequence(text)
    tokens=[dict1[word] if word in dict1 else 0 for word in tokens]
    return sequence.pad_sequences([tokens],1000,padding="post",truncating="post")[0]

## Decoding Function

In [6]:
reverse_word_index = {value: key for (key, value) in dict1.items()}

def decoding(integers):
    PAD=0
    text=""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "
    return text[:-1]

## One Hot Encode labels

In [40]:
le = LabelEncoder()
y=le.fit_transform(df["Assignment group"])

In [41]:
lb=np.array([0,1,2,3,4,5,6,7,8,9])
lb=dict(zip(le.classes_, lb))
y=tf.keras.utils.to_categorical(y, num_classes=10)

### Shape of X and y

In [3]:
print(X.shape,y.shape)
le.classes_

## Model and compilation

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(8000, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          256000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                16640     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 275,050
Trainable params: 275,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.1,
                                                    shuffle=True,
                                                    random_state=6)

## Training the Model

In [18]:
batch_size=32
epochs=7
model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,verbose=2, validation_split=.3)

C:\Users\PC-ASUS\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 13717 samples, validate on 5880 samples
Epoch 1/7
 - 856s - loss: 1.3957 - acc: 0.5614 - val_loss: 1.0776 - val_acc: 0.6588
Epoch 2/7
 - 895s - loss: 0.9472 - acc: 0.7006 - val_loss: 0.9509 - val_acc: 0.6906
Epoch 3/7
 - 818s - loss: 0.7802 - acc: 0.7411 - val_loss: 0.8413 - val_acc: 0.7286
Epoch 4/7
 - 820s - loss: 0.6531 - acc: 0.7882 - val_loss: 0.7539 - val_acc: 0.7611
Epoch 5/7
 - 831s - loss: 0.5515 - acc: 0.8242 - val_loss: 0.7350 - val_acc: 0.7670
Epoch 6/7
 - 806s - loss: 0.4665 - acc: 0.8531 - val_loss: 0.7474 - val_acc: 0.7741
Epoch 7/7
 - 809s - loss: 0.4052 - acc: 0.8719 - val_loss: 0.7215 - val_acc: 0.7872


## Model Evaluation

In [20]:
results= model.evaluate(X_test,y_test)
print(results)

2178/2178 [==============================] - 11s 5ms/step
[0.7718274645349977, 0.7699724517906336]


## Saving the Model(Arch.) and its weights

In [21]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:
model_new = load_model('model.h5')

## Making prediction on new Incidents

In [8]:
def check(text):
    encoded_text= encoding(text)
    pred = np.zeros((1,1000))
    pred[0]=encoded_text
    result = model_new.predict(pred)
    print (result[0])

In [6]:
check("URL block request")

In [5]:
le.classes_